In [1]:
import spacy
from nltk.corpus import wordnet as wn
import random

In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
def synonyms_list(sentence):
    doc = nlp(sentence)
    result = []
    for index, token in enumerate(doc):
        currentSynonyms = set([])

        if token.pos_ == 'NOUN':
            pos = wn.NOUN
        elif token.pos_ == 'VERB':
            pos = wn.VERB
        elif token.pos_ == 'ADV':
            pos = wn.ADV
        elif token.pos_ == 'ADJ':
            pos = wn.ADJ
        else:
            result.append(currentSynonyms)
            continue
        for synset in wn.synsets(token.text, pos):
            for lemma in synset.lemmas():
                currentSynonyms.add(lemma.name())
        result.append(currentSynonyms)
    return result

In [5]:
def sentence_similarity(sentence1, sentence2):
    search_doc = nlp(sentence1)
    main_doc = nlp(sentence2)
    search_doc_no_stop_words = nlp(' '.join([str(t.lemma_) for t in search_doc if not t.is_stop]))
    main_doc_no_stop_words = nlp(' '.join([str(t.lemma_) for t in main_doc if not t.is_stop]))
    return (search_doc_no_stop_words.similarity(main_doc_no_stop_words))

In [6]:
def paraphraseSentence(sentence, temperature):
    synonyms = synonyms_list(sentence)
    probability = temperature / 5
    threshold_number = int(probability * 100)
    threshold_accuracy = 1 - (temperature / 10)
    doc = nlp(sentence)
    epochs = 100
    while (epochs >= 0):
        tokens = [token.text for token in doc]
        for index, token in enumerate(tokens):
            if (synonyms[index] and random.randint(1, 100) <= threshold_number):
                tokens[index] = random.sample(synonyms[index], 1)[0]
        new_sentence = ' '.join(tokens).replace('_',' ')
        if (sentence_similarity(new_sentence,sentence) >= threshold_accuracy):
            return new_sentence
    return sentence

In [9]:
def model1(sentence, temperature):
    return paraphraseSentence(sentence, temperature)

In [11]:
from translate import Translator
import translators as ts

Using Bulgaria server backend.


In [16]:
def model2(sentence, provider, from_language, to_language):
    if provider == 'iciba':
        translate_to = ts.iciba(sentence, from_language = from_language, to_language = to_language)
        translate_back = ts.iciba(translate_to, from_language = to_language, to_language = from_language)
        return translate_back
    elif provider == 'yandex':
        translate_to = ts.yandex(sentence, from_language = from_language, to_language = to_language)
        translate_back = ts.yandex(translate_to, from_language = to_language, to_language = from_language)
        return translate_back
    elif provider == 'argos':
        translate_to = ts.argos(sentence, from_language = from_language, to_language = to_language)
        translate_back = ts.argos(translate_to, from_language = to_language, to_language = from_language)
        return translate_back

In [20]:
from nltk import Tree

In [34]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.text, [to_nltk_tree(child) for child in node.children])
    else:
        return node.text

In [26]:
def draw_tree(sentence):
    doc = nlp(sentence)
    [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]
    return

In [39]:
def tree_to_list(root, result):
    if not root:
        return
    result.append(root.text)
    for child in root.children:
        tree_to_list(child, result)

In [40]:
def order_phrase_from_sentence(text_sentence, phrase_list):
    result = []
    for word in text_sentence:
        if (word in phrase_list):
            result.append(word)
    return result

In [42]:
def model3(sentence):
    doc = nlp(sentence)
    parsed_sentence = list(doc.sents)[0]
    text_sentence = [token.text for token in parsed_sentence]
    phrases_list = [[parsed_sentence.root.text]]
    for child in parsed_sentence.root.children:
        temp_list = []
        tree_to_list(child, temp_list)
        phrases_list.append(temp_list)
    ordered_phrases = [order_phrase_from_sentence(text_sentence, phrase) for phrase in phrases_list]
    reordered_phrases = random.sample(ordered_phrases, len(ordered_phrases))
    final_sentence = []
    for phrase in reordered_phrases:
        final_sentence += phrase
    final_sentence_to_lower = [word.lower() for word in final_sentence]
    final_sentence_to_lower[0] = final_sentence_to_lower[0].capitalize()
    draw_tree(sentence)
    return (' ').join(final_sentence_to_lower)

In [10]:
model1('My dog likes playing in the water.', 2)

'My dog wish playacting in the water .'

In [47]:
model1('My dog likes playing in the water.', 4)

'My hot dog like playing in the water supply .'

In [48]:
model1('My dog likes playing in the water', 4)

'My dog care playing in the water system'

In [52]:
model1('My dog likes playing in the water', 5)

'My hound wish playacting in the pee'

In [56]:
model1('The future belongs to those who believe in the beauty of their dreams',5)

'The time to come go to those who think in the smasher of their dream'

In [63]:
model1('The future belongs to those who believe in the beauty of their dreams',5)

'The hereafter belong to to those who trust in the looker of their aspiration'

In [17]:
model2('My dog likes playing in the water.', 'iciba', 'en', 'ru')

'My dog loves playing in the water.'

In [18]:
model2('My dog likes playing in the water.', 'yandex', 'en', 'ru')

'My dog likes to play in the water.'

In [72]:
model2('The future belongs to those who believe in the beauty of their dreams', 'iciba', 'en', 'bg')

'The future belongs to those who believe in beautiful dreams'

In [74]:
model2('The future belongs to those who believe in the beauty of their dreams', 'argos', 'en', 'hi')

'The future relates to those who believe in the beauty of their dreams'

In [75]:
model2('The future belongs to those who believe in the beauty of their dreams', 'argos', 'en', 'zh')

'The future belongs to those who believe that their dream is defended.'

In [84]:
model3('My dog likes playing in the water')

    likes        
  ____|______     
 |        playing
 |           |    
 |           in  
 |           |    
dog        water 
 |           |    
 My         the  



'Playing in the water my dog likes'

In [86]:
model3('The future belongs to those who believe in the beauty of their dreams')

           belongs                      
   ___________|_______                   
  |                   to                
  |                   |                  
  |                 those               
  |                   |                  
  |                believe              
  |      _____________|______            
  |     |                    in         
  |     |                    |           
  |     |                  beauty       
  |     |              ______|______     
  |     |             |             of  
  |     |             |             |    
future  |             |           dreams
  |     |             |             |    
 The   who           the          their 



'The future to those who believe in the beauty of their dreams belongs'

In [94]:
model3('They watched the movie on Friday')

     watched       
  ______|______     
 |    movie    on  
 |      |      |    
They   the   Friday



'On friday they watched the movie'